In [57]:
import pandas as pd
import numpy as np
import re
from scipy import sparse
from sklearn import preprocessing, linear_model
import sklearn.feature_extraction.text as txt

def clean(s):
    try:
        return " ".join(re.findall(r'\w+', s)).lower()
    except:
        
        return " ".join(re.findall(r'\w+', "no_text")).lower()

train = pd.read_csv('TrainingData.csv')
test = pd.read_csv('TestData.csv')
sample = pd.read_csv('SubmissionFormat.csv')

training_data = train[['FTE','Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4', 'Total']]

cols = ['Function','Object_Type','Operating_Status','Position_Type','Pre_K', 'Reporting',
                'Sharing','Student_Type', 'Use']

labels = train[cols]


In [58]:

it=['Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4']
test_data = test[['FTE','Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4', 'Total']]


label_encoder = preprocessing.LabelEncoder()
for i in cols:
    labels[i] = label_encoder.fit_transform(np.array(labels[i]))
    
    
training_data = training_data.drop('FTE', axis = 1)
training_data = training_data.drop('Total', axis = 1)

test_data = test_data.drop('FTE', axis = 1)
test_data = test_data.drop('Total', axis = 1)
#mans.csv created
for i in it:
        training_data[i] = training_data[i].apply(clean)
for i in it:
        test_data[i] = test_data[i].apply(clean)
        
cleaned=training_data
cleaned.to_csv('cleaned.csv', index = False)

C:\Users\derri\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [75]:
labels['Function']

0         34
1         18
2         34
3         33
4         33
5          8
6         14
7         11
8         34
9         33
10        18
11        14
12        34
13        18
14        18
15        33
16        14
17        28
18        36
19        18
20        18
21        33
22        34
23        18
24        33
25        34
26        18
27        33
28        34
29        32
          ..
400247    30
400248    18
400249    33
400250    34
400251    34
400252    33
400253     0
400254    34
400255     0
400256    34
400257    23
400258    18
400259    18
400260    33
400261    26
400262    34
400263    34
400264    18
400265    32
400266    14
400267    18
400268    33
400269    11
400270    18
400271    34
400272    23
400273    33
400274    21
400275    17
400276    33
Name: Function, Length: 400277, dtype: int32

In [60]:
training_data["combined"] = [' '.join(row) for row in training_data[training_data.columns].values]

In [62]:
test_data["combined"] = [' '.join(row) for row in test_data[test_data.columns].values]


In [68]:

tfidf = txt.TfidfVectorizer(ngram_range=(1, 2), max_df=1.0, min_df=10)
hsv = txt.HashingVectorizer()


tfidf.fit(training_data['combined'])
hsv.fit(test_data['combined'])


HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
                  decode_error='strict', dtype=<class 'numpy.float64'>,
                  encoding='utf-8', input='content', lowercase=True,
                  n_features=1048576, ngram_range=(1, 1), norm='l2',
                  preprocessor=None, stop_words=None, strip_accents=None,
                  token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None)

In [77]:
training_data['combined']

0         no_text no_text general fund teacher elementar...
1         no_text rgn gob no_text blank no_text contract...
2         no_text no_text general purpose school tcher 2...
3         no_text unalloc budgets schools no_text teache...
4         no_text non project no_text teacher secondary ...
5         no_text non project no_text custodian pt jobs ...
6         no_text no_text local fund no_text no_text edu...
7         no_text non project no_text sub manager food s...
8         no_text ela s teaching spanish only no_text te...
9         no_text unalloc budgets schools no_text teache...
10        no_text state and federal projects coordinatio...
11        no_text instruction general fund no_text no_te...
12        position control pools no_text general purpose...
13        no_text disadvantaged youth title i disadvanta...
14        no_text basic fefp k 12 conversion charter sch...
15        all campus payroll instruction general operati...
16        no_text basic fefp k 12 genera

In [70]:




X_tfidf = tfidf.ttfidfransform(training_data['combined'])

X_test_tfidf = tfidf.transform(test_data['combined'])

X_hsv = hsv.transform(training_data['combined'])
X_test_hsv = hsv.transform(test_data['combined'])

X = sparse.hstack((X_hsv, X_tfidf))
X_test = sparse.hstack((X_test_hsv, X_test_tfidf))



AttributeError: 'TfidfVectorizer' object has no attribute 'ttfidfransform'

In [40]:
preds1 = []
preds2 = []
preds3 = []
preds4 = []
for i in range(len(cols)):
    print ("label = ", i)
    sgd1 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, alpha = 0.000001)
    sgd2 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, penalty = 'l1')
    sgd3 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, penalty = 'l2')
    sgd4 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, penalty = 'elasticnet')
    sgd1.fit(X, labels.iloc[:,i])
    sgd2.fit(X, labels.iloc[:,i])
    sgd3.fit(X, labels.iloc[:,i])
    sgd4.fit(X, labels.iloc[:,i])
    if i == 0:
        preds1 = sgd1.predict_proba(X_test)
        preds2 = sgd2.predict_proba(X_test)
        preds3 = sgd3.predict_proba(X_test)
        preds4 = sgd4.predict_proba(X_test)
    else:
        print(len(preds1),len(sgd1.predict_proba(X_test)))
        preds1 = np.hstack((preds1,sgd1.predict_proba(X_test)))
        preds2 = np.hstack((preds2,sgd2.predict_proba(X_test)))
        preds3 = np.hstack((preds3,sgd3.predict_proba(X_test)))
        preds4 = np.hstack((preds4,sgd4.predict_proba(X_test)))
preds = (preds1 + preds2 + preds3 + preds4)/4.0

for i in range(1, len(sample.columns)):
    colname = sample.columns[i]
    sample[str(colname)] = preds[:,i-1]

sample.to_csv('Logg_Model.csv', index = False)

label =  0
label =  1
50064 50064
label =  2
50064 50064
label =  3
50064 50064
label =  4
50064 50064
label =  5
50064 50064
label =  6
50064 50064
label =  7
50064 50064
label =  8
50064 50064
